Similarly, remove all movies with less than 5 actors/actresses on list, construct
a movie network according to the set of actors/actresses, with weight assigned
as the jaccard index of the actor sets of 2 movies. Now we have an undirected
network instead.

In [1]:
from tqdm import tqdm_notebook as timer
import re
import string
from collections import defaultdict

In [2]:
A = {}
M = {}

def clean_string(s):
    # Remove (*)
    s = re.sub(r'\([^0-9]+\)', '', s)

    # Remove {*}
    s = re.sub(r'\{.*\}', '', s)
    
    # Remove multiple spaces
    s = re.sub(r'\s+', ' ', s)

    # Strip spaces etc
    s = s.lstrip()
    s = s.rstrip()

    return s

with open("movie_map.txt", "r") as f:
    for line in timer(f, total = 30396, desc="movie_map"):
        splits = line.split("\t\t")
        
        actor_name = splits[0]
        
        actor_name = splits[0]
        movies = set(map(clean_string, splits[1:]))
        
        if actor_name in A:
            A[actor_name] = A[actor_name].union(movies)
        else:
            A[actor_name] = movies

        for movie in movies:
            if movie not in M:
                M[movie] = [actor_name]
            else:
                M[movie].append(actor_name)

In [3]:
M = {movie : set(actors) for (movie, actors) in M.iteritems()}

# Create a movie network

In [4]:
with open("movie_network.txt", "w") as f:
    for (m1, actors) in timer(M.iteritems(), total=len( M.keys() )):
        edges = set()
        
        for a in actors:
            for m2 in A[a]:
                common = M[m1].intersection(M[m2])
                
                w = float( len( common ) ) / (len(M[m1]) + len(M[m2]) + len(common))
                
                edges.add( (m2, w) )
        
        for (m2, w) in edges:
            f.write("%s\t%s\t%f\n" % (m1, m2, w))

In [5]:
%%bash
# number of edges
wc -l movie_network.txt

 55035189 movie_network.txt


# Process the movie genres

In [6]:
G = {}
printable = set(string.printable)

def clean_string(s):
    # Remove (*)
    s = re.sub(r'\([^0-9]+\)', '', s)

    # Remove {*}
    s = re.sub(r'\{.*\}', '', s)
    
    # Remove multiple spaces
    s = re.sub(r'\s+', ' ', s)

    # Strip spaces etc
    s = s.lstrip()
    s = s.rstrip()

    return s

with open("data/movie_genre.txt") as f:
    for line in timer(f, total=1010991, desc="Movie Genres"):
        line = filter(lambda x : x in printable, line.decode('latin1')).encode('ascii')
        
        splits = line.split('\t\t')
        movie = clean_string(splits[0])
        genre = clean_string(splits[1])
        
        if movie in M:
            G[movie] = genre

In [7]:
# For unidentified movies 
for m in set(M.keys()).difference(set(G.keys())):
    G[m] = "Unknown"

In [8]:
with open("genres.txt", "w") as f:
    for (movie, genre) in G.iteritems():
        f.write("%s\t%s\n" % (movie, genre))

# Write graph in Metis format

In [9]:
M = {movie : set(actors) for (movie, actors) in M.iteritems()}

In [10]:
vmap = {movie : i + 1 for (i, movie) in enumerate(M.keys())}
rmap = {i + 1 : movie for (i, movie) in enumerate(M.keys())}

In [11]:
edges = defaultdict(set)

for (m1, actors) in timer(M.iteritems(), total=len( M.keys() )):
    u = vmap[m1]

    for a in actors:
        for m2 in A[a]:
            v = vmap[m2]

            if u != v and u not in edges[v]:
                common = M[m1].intersection(M[m2])
                
                # Metis cannot handle fractional weights. WTF?? 
                w = (len( common ) * 1000) / (len(M[m1]) + len(M[m2]) + len(common))
                edges[u].add( (v, w) )

35537/|/ 70%|| 35537/50833 [03:40<01:34, 161.25it/s]


In [12]:
len( edges.keys() )

50833

In [15]:
with open("metis/graph.txt", "w") as f:
    f.write("%d %d %d\n" % ( len( edges.keys() ),  sum( map(len, edges.values()) ) / 2, 1)) # Replace with number of edges
    for m in M.keys():
        f.write(' '.join(map(lambda x : str(x[0]) + " " + str(x[1]), edges[vmap[m]])) + "\n")

In [18]:
%%bash

# Run metis on the metis graph file generated in Part 4
gpmetis metis/graph.txt 2

******************************************************************************
METIS 5.0 Copyright 1998-13, Regents of the University of Minnesota
 (HEAD: , Built on: Oct 13 2016, 16:46:05)
 size of idx_t: 32bits, real_t: 32bits, idx_t *: 64bits

Graph Information -----------------------------------------------------------
 Name: metis/graph.txt, #Vertices: 50833, #Edges: 27492178, #Parts: 2

Options ---------------------------------------------------------------------
 ptype=kway, objtype=cut, ctype=shem, rtype=greedy, iptype=metisrb
 dbglvl=0, ufactor=1.030, no2hop=NO, minconn=NO, contig=NO, nooutput=NO
 seed=-1, niter=10, ncuts=1

Direct k-way Partitioning ---------------------------------------------------
 - Edgecut: 8591371, communication volume: 38287.

 - Balance:
     constraint #0:  1.002 out of 0.000

 - Most overweight partition:
     pid: 1, actual: 25461, desired: 25416, ratio: 1.00.

 - Subdomain connectivity: max: 1, min: 1, avg: 1.00

 - Each partition is contiguous.

